
Notebook de création du dataset<br/>
Work in progress: voir si utilisation du 2e notebook Kaggle, et il reste à décider de quelles variables utiliser à la fois pour les critères que pour les cotes

# 1. Données équipes par match

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# uploader le kaggle.json préalablement récupéré
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"mllacc","key":"9b91572c1d94b3d2a3ad803255ddc043"}'}

In [1]:
# Créer un dossier kaggle
! mkdir ~/.kaggle
# Copie le fichier kaggle.json dans le dossier kaggle
! cp kaggle.json ~/.kaggle/
# Change les permisions du fichier kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [4]:
# données kaggle https://www.kaggle.com/shubhmamp/english-premier-league-match-data
! kaggle datasets download -d shubhmamp/english-premier-league-match-data

  0% 0.00/5.57M [00:00<?, ?B/s]
100% 5.57M/5.57M [00:00<00:00, 51.1MB/s]


In [6]:
# creation d'un folder dataset dans le drive, et extraction des données
! mkdir '/content/drive/My Drive/dataset'
! unzip english-premier-league-match-data.zip -d '/content/drive/My Drive/dataset'

Archive:  english-premier-league-match-data.zip
  inflating: /content/drive/My Drive/dataset/datafile/season14-15/season_match_stats.json  
  inflating: /content/drive/My Drive/dataset/datafile/season14-15/season_stats.json  
  inflating: /content/drive/My Drive/dataset/datafile/season15-16/season_match_stats.json  
  inflating: /content/drive/My Drive/dataset/datafile/season15-16/season_stats.json  
  inflating: /content/drive/My Drive/dataset/datafile/season16-17/season_match_stats.json  
  inflating: /content/drive/My Drive/dataset/datafile/season16-17/season_stats.json  
  inflating: /content/drive/My Drive/dataset/datafile/season17-18/season_match_stats.json  
  inflating: /content/drive/My Drive/dataset/datafile/season17-18/season_stats.json  
  inflating: /content/drive/My Drive/dataset/datafilev2/datafile/season14-15/season_match_stats.json  
  inflating: /content/drive/My Drive/dataset/datafilev2/datafile/season14-15/season_stats.json  
  inflating: /content/drive/My Drive/dat

In [7]:
import json
import pandas as pd

process basé sur l'exemple disponible dans kaggle: https://www.kaggle.com/maximerig/create-useful-dataframes-from-json-files

In [8]:
stats_14_15_json = json.load(open('/content/drive/My Drive/dataset/datafilev2/datafile/season14-15/season_stats.json'))
stats_15_16_json = json.load(open('/content/drive/My Drive/dataset/datafilev2/datafile/season15-16/season_stats.json'))
stats_16_17_json = json.load(open('/content/drive/My Drive/dataset/datafilev2/datafile/season16-17/season_stats.json'))
stats_17_18_json = json.load(open('/content/drive/My Drive/dataset/datafilev2/datafile/season17-18/season_stats.json'))

In [12]:
def home_team_stats(json):
    stats = pd.DataFrame()
    
    row = 0
    for match_id, infos_match in json.items():
        stats.loc[row, 'match_id'] = match_id
        home = dict(list(infos_match.values())[0])
        for column, team_info in home['team_details'].items():
            stats.loc[row, column] = team_info
        for column, team_stat in home['aggregate_stats'].items():
            stats.loc[row, column] = team_stat
        row += 1
    
    #stats = stats.fillna(0)
    stats['date'] = pd.to_datetime(stats['date'], dayfirst=True)
    stats = stats.sort_values(by=['date', 'match_id'])
    stats = stats.reset_index(drop=True)
    
    stats['team_rating'] = stats['team_rating'].astype(float)
    for column in stats.columns[5:]:
        stats[column] = stats[column].astype(float)
    
    return stats

In [13]:
def away_team_stats(json):
    stats = pd.DataFrame()
    
    row = 0
    for match_id, infos_match in json.items():
        stats.loc[row, 'match_id'] = match_id
        away = dict(list(infos_match.values())[1])
        for column, team_info in away['team_details'].items():
            stats.loc[row, column] = team_info
        for column, team_stat in away['aggregate_stats'].items():
            stats.loc[row, column] = team_stat
        row += 1
    
    #stats = stats.fillna(0)
    stats['date'] = pd.to_datetime(stats['date'], dayfirst=True)
    stats = stats.sort_values(by=['date', 'match_id'])
    stats = stats.reset_index(drop=True)
    
    stats['team_rating'] = stats['team_rating'].astype(float)
    for column in stats.columns[5:]:
        stats[column] = stats[column].astype(float)
    
    return stats

In [14]:
team_stats_home = {'season_14_15': home_team_stats(stats_14_15_json).add_prefix('home_'), 
                   'season_15_16': home_team_stats(stats_15_16_json).add_prefix('home_'), 
                   'season_16_17': home_team_stats(stats_16_17_json).add_prefix('home_'),  
                   'season_17_18': home_team_stats(stats_17_18_json).add_prefix('home_')}

team_stats_away = {'season_14_15': away_team_stats(stats_14_15_json).add_prefix('away_'), 
                   'season_15_16': away_team_stats(stats_15_16_json).add_prefix('away_'), 
                   'season_16_17': away_team_stats(stats_16_17_json).add_prefix('away_'),  
                   'season_17_18': away_team_stats(stats_17_18_json).add_prefix('away_')}

In [15]:
# concatenation pour toutes les données home
df_home = pd.concat([team_stats_home['season_14_15'],team_stats_home['season_15_16'],team_stats_home['season_16_17'],team_stats_home['season_17_18']])

In [16]:
# concatenation pour toutes les données away
df_away = pd.concat([team_stats_away['season_14_15'],team_stats_away['season_15_16'],team_stats_away['season_16_17'],team_stats_away['season_17_18']])

In [17]:
# concatenation home et away
df_merge = df_home.merge(df_away, left_on=['home_match_id'], right_on=['away_match_id'])

In [18]:
df_merge = df_merge.rename(columns={"home_match_id": "match_id", "home_date": "date"})

In [19]:
# il semblerait dans le dataset que pour les colonnes de résultats (home_goals, away_goals) les Nan équivalent à 0 (pas de goal marqué)

In [20]:
df_merge['home_goals'] = df_merge['home_goals'].fillna(0)

In [21]:
df_merge['away_goals'] = df_merge['away_goals'].fillna(0)

In [ ]:
# ajout de l'issue des matches (H => home, A => Away, D => Draw)

In [22]:
def get_match_result(row):
  home_goals = row['home_goals']
  away_goals = row['away_goals']
  winner = 0
  if (home_goals > away_goals):
    winner = 'H'
  elif (away_goals > home_goals):
    winner = 'A'
  elif (home_goals == away_goals):
    winner = 'D'
  return winner

In [23]:
# ajout du résultat du match
df_merge['match_result'] = df_merge.apply(lambda row: get_match_result(row), axis = 1)

# 2. Cotes

In [24]:
! pip install datapackage

     |████████████████████████████████| 85 kB 5.0 MB/s 
     |████████████████████████████████| 72 kB 593 kB/s 
     |████████████████████████████████| 68 kB 6.6 MB/s 
     |████████████████████████████████| 45 kB 3.5 MB/s 
     |████████████████████████████████| 126 kB 52.2 MB/s 
     |████████████████████████████████| 243 kB 49.7 MB/s 
     |████████████████████████████████| 131 kB 45.4 MB/s 
     |████████████████████████████████| 8.0 MB 52.3 MB/s 
     |████████████████████████████████| 79 kB 8.2 MB/s 
     |████████████████████████████████| 127 kB 60.5 MB/s 
  Created wheel for botocore: filename=botocore-1.21.17-py3-none-any.whl size=7791094 sha256=abb5ffd471deddac497f44d0841ddd7dcb5b17087b4719cdb54a74c2d5e2405b
  Stored in directory: /root/.cache/pip/wheels/88/7d/d8/11789e759e5fb1665dd3bf30e77c66dbd76a6f8b51e9b18eae
  Created wheel for linear-tsv: filename=linear_tsv-1.1.0-py3-none-any.whl size=7399 sha256=a6d8b883e823d10b44f5861e9899ac2441dad981e960109607e28b078d43ccd7
  Stored

In [ ]:
import datapackage

In [ ]:
#utilisation du process indiqué sur le site des données

data_url = 'https://datahub.io/sports-data/english-premier-league/datapackage.json'

# to load Data Package into storage
package = datapackage.Package(data_url)

# to load only tabular data
resources = package.resources

cotes_1415 = pd.read_csv(resources[5].descriptor['path'])
cotes_1516 = pd.read_csv(resources[4].descriptor['path'])
cotes_1617 = pd.read_csv(resources[3].descriptor['path']) 
cotes_1718 = pd.read_csv(resources[2].descriptor['path']) 

In [ ]:
cotes = pd.concat([cotes_1415, cotes_1516, cotes_1617, cotes_1718])

In [ ]:
# je laisse la colonne d'origine pour verifications ulterieures
cotes['date'] = cotes['Date'].apply(lambda x: pd.to_datetime(x))

# 3. Jointure données matches et cotes

In [ ]:
# revoir les noms des colonnes
# df_merge.info()

In [ ]:
df_stats_cotes = df_merge.merge(cotes, left_on = ['date', 'home_team_name', 'away_team_name'], right_on = ['date', 'HomeTeam', 'AwayTeam'])

In [ ]:
df_stats_cotes.to_csv('df_stats_cotes.csv')